In [2]:
# torch-neuronx requires 1.13.1 or more above

!pip install -U pip --quiet
#!pip install --upgrade google-auth --quiet

!pip install protobuf==3.19.6 --quiet
!pip install pytorch-lightning==1.6.4 --quiet
!pip install transformers==4.11.3 --quiet
!pip install torchvision==0.14.1 --quiet
!pip install torch==1.13.1 --quiet
!pip install timm==0.5.4 --quiet
!pip install donut-python==1.0.9 --quiet
!pip config set global.extra-index-url https://pip.repos.neuron.amazonaws.com
!pip install neuronx-cc==2.* torch-neuronx --quiet

!pip install PyYAML==5.4.1 --quiet
!pip install --upgrade requests chardet --quiet
!pip install --upgrade urllib3==1.26.7 --quiet

Writing to /home/ubuntu/.config/pip/pip.conf


In [3]:
import transformers

In [4]:
!dpkg -l aws-neuronx-runtime-lib

Desired=Unknown/Install/Remove/Purge/Hold
| Status=Not/Inst/Conf-files/Unpacked/halF-conf/Half-inst/trig-aWait/Trig-pend
|/ Err?=(none)/Reinst-required (Status,Err: uppercase=bad)
||/ Name                    Version            Architecture Description
+++-=======================-==================-============-=================================
ii  aws-neuronx-runtime-lib 2.13.6.0-29de104d6 amd64        neuron_runtime built using CMake


In [5]:
!pip list |grep -e torch -e neuron -e donut -e transformers

aws-neuronx-runtime-discovery 2.9
donut-python                  1.0.9
libneuronxla                  0.5.207
neuronx-cc                    2.6.0.19+3d819e565
neuronx-hwm                   2.6.0.0+826e77395
pytorch-lightning             1.6.4
torch                         1.13.1
torch-neuronx                 1.13.1.1.7.0
torch-xla                     1.13.1+torchneuron6
torchmetrics                  0.11.4
torchvision                   0.14.1
transformers                  4.11.3


In [6]:
import datetime
import subprocess

# 現在の日時を取得
now = datetime.datetime.now()
date_str = now.strftime('%Y%m%d_%H%M%S')

# Pythonバージョンの確認
python_version = subprocess.run(['python3.8', '--version'], stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True).stdout.strip()

# CUDAバージョンの確認
try:
    nvcc_version = subprocess.run(['nvcc', '--version'], stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True).stdout.strip()
    _, cuda_version = nvcc_version.split('\n')[-2].split(',')[1].split()
except:
    cuda_version = 'CPU'

# torchバージョンの確認
import torch
torch_version = torch.__version__

# transformersバージョンの確認
import transformers
transformers_version = transformers.__version__

# 実行時のコマンドと実行結果を保存するファイル名
log_file = f"diffusion_{date_str}.log"


print(f"Python version: {python_version}\n")
print(f"CUDA version: {cuda_version}\n")
print(f"torch version: {torch_version}\n")
print(f"transformers version: {transformers_version}\n\n")

Python version: Python 3.8.10

CUDA version: 11.8

torch version: 1.13.1+cu117

transformers version: 4.11.3




In [7]:
import torch
from PIL import Image
import sys
from donut import DonutModel

import IPython
import pprint
from donut.model import DonutConfig, DonutModel, SwinEncoder, BARTDecoder

In [8]:
# https://github.com/clovaai/donut/blob/a0e94bf145d81cfc934eae8848f1269ae9ca46a2/donut/model.py#L11
import math
import os
import re
from typing import Any, List, Optional, Union

import numpy as np
import PIL
import timm
import torch
import torch.nn as nn
import torch.nn.functional as F
from PIL import ImageOps
from timm.data.constants import IMAGENET_DEFAULT_MEAN, IMAGENET_DEFAULT_STD
from timm.models.swin_transformer import SwinTransformer
from torchvision import transforms
from torchvision.transforms.functional import resize, rotate
from transformers import MBartConfig, MBartForCausalLM, XLMRobertaTokenizer
from transformers.file_utils import ModelOutput
from transformers.modeling_utils import PretrainedConfig, PreTrainedModel

In [9]:
import torch
import torch_neuronx
import os
import numpy as np
from torch.nn import functional as F

In [10]:
import torch
from PIL import Image
import sys

import IPython
import pprint

In [11]:
from donut import DonutModel

In [12]:
!rm -rf test_01.jpg

# 英語のスライド
!wget -c https://media.slidesgo.com/storage/162635/conversions/1-market-share-infographics-thumb.jpg \
      -O ./test_01.jpg

# 日本語のレシート
# !wget -c https://www.isp21.co.jp/wp-content/uploads/solution/library/library02-1.jpg \
#       -O ./test_01.jpg

input_img_path = './test_01.jpg'

--2023-06-02 12:25:45--  https://media.slidesgo.com/storage/162635/conversions/1-market-share-infographics-thumb.jpg
Resolving media.slidesgo.com (media.slidesgo.com)... 172.67.9.18, 104.22.0.146, 104.22.1.146, ...
Connecting to media.slidesgo.com (media.slidesgo.com)|172.67.9.18|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 23066 (23K) [image/jpeg]
Saving to: ‘./test_01.jpg’

./test_01.jpg       100%[===================>]  22.53K  --.-KB/s    in 0s      

2023-06-02 12:25:45 (205 MB/s) - ‘./test_01.jpg’ saved [23066/23066]



In [13]:
donut_model = DonutModel.from_pretrained("naver-clova-ix/donut-base-finetuned-rvlcdip").to("cpu")
donut_model.eval() 

Downloading:   0%|          | 0.00/401 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/965M [00:00<?, ?B/s]

/home/ubuntu/aws_neuron_venv_pytorch/lib/python3.8/site-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3190.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


Downloading:   0%|          | 0.00/1.24M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/558 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/747 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/536 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/3.83M [00:00<?, ?B/s]

DonutModel(
  (encoder): SwinEncoder(
    (model): SwinTransformer(
      (patch_embed): PatchEmbed(
        (proj): Conv2d(3, 128, kernel_size=(4, 4), stride=(4, 4))
        (norm): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
      )
      (pos_drop): Dropout(p=0.0, inplace=False)
      (layers): Sequential(
        (0): BasicLayer(
          dim=128, input_resolution=(640, 480), depth=2
          (blocks): ModuleList(
            (0): SwinTransformerBlock(
              (norm1): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
              (attn): WindowAttention(
                (qkv): Linear(in_features=128, out_features=384, bias=True)
                (attn_drop): Dropout(p=0.0, inplace=False)
                (proj): Linear(in_features=128, out_features=128, bias=True)
                (proj_drop): Dropout(p=0.0, inplace=False)
                (softmax): Softmax(dim=-1)
              )
              (drop_path): Identity()
              (norm2): LayerNorm((128,), ep

In [14]:
task_prompt = "<s_rvlcdip>"

input_img = Image.open("./test_01.jpg")
output = donut_model.inference(image=input_img, prompt=task_prompt)["predictions"][0]

In [15]:
output

{'class': 'presentation'}

In [18]:
model = SwinTransformer() 

In [19]:
swin_state_dict = timm.create_model("swin_base_patch4_window12_384", pretrained=True).state_dict()

Downloading: "https://github.com/SwinTransformer/storage/releases/download/v1.0.0/swin_base_patch4_window12_384_22kto1k.pth" to /home/ubuntu/.cache/torch/hub/checkpoints/swin_base_patch4_window12_384_22kto1k.pth


In [20]:
new_swin_state_dict = model.state_dict()

In [21]:
model.load_state_dict(new_swin_state_dict)

<All keys matched successfully>

In [22]:
from timm.models.swin_transformer import WindowAttention, SwinTransformer

In [23]:
window_attention = WindowAttention(dim=1024, num_heads=32, window_size=(10, 10), qkv_bias=True, attn_drop=0.0, proj_drop=0.0) 

In [24]:
dummy_input = torch.zeros([48, 100, 1024])

In [25]:
window_attention.forward(dummy_input) 

tensor([[[ 0.0102, -0.0268, -0.0262,  ..., -0.0072, -0.0084,  0.0367],
         [ 0.0102, -0.0268, -0.0262,  ..., -0.0072, -0.0084,  0.0367],
         [ 0.0102, -0.0268, -0.0262,  ..., -0.0072, -0.0084,  0.0367],
         ...,
         [ 0.0102, -0.0268, -0.0262,  ..., -0.0072, -0.0084,  0.0367],
         [ 0.0102, -0.0268, -0.0262,  ..., -0.0072, -0.0084,  0.0367],
         [ 0.0102, -0.0268, -0.0262,  ..., -0.0072, -0.0084,  0.0367]],

        [[ 0.0102, -0.0268, -0.0262,  ..., -0.0072, -0.0084,  0.0367],
         [ 0.0102, -0.0268, -0.0262,  ..., -0.0072, -0.0084,  0.0367],
         [ 0.0102, -0.0268, -0.0262,  ..., -0.0072, -0.0084,  0.0367],
         ...,
         [ 0.0102, -0.0268, -0.0262,  ..., -0.0072, -0.0084,  0.0367],
         [ 0.0102, -0.0268, -0.0262,  ..., -0.0072, -0.0084,  0.0367],
         [ 0.0102, -0.0268, -0.0262,  ..., -0.0072, -0.0084,  0.0367]],

        [[ 0.0102, -0.0268, -0.0262,  ..., -0.0072, -0.0084,  0.0367],
         [ 0.0102, -0.0268, -0.0262,  ..., -0

In [26]:
torch.jit.trace(window_attention, dummy_input) 

WindowAttention(
  original_name=WindowAttention
  (qkv): Linear(original_name=Linear)
  (attn_drop): Dropout(original_name=Dropout)
  (proj): Linear(original_name=Linear)
  (proj_drop): Dropout(original_name=Dropout)
  (softmax): Softmax(original_name=Softmax)
)

In [27]:
torch_neuronx.trace(window_attention, dummy_input) 

NeuronModule(original_name=NeuronModule)

In [28]:
from timm.models.swin_transformer import SwinTransformerBlock

In [29]:
stb = SwinTransformerBlock(dim=1024, input_resolution=(80, 60), num_heads=32, window_size=10, shift_size=5, mlp_ratio=4.0, qkv_bias=True, drop=0.0, attn_drop=0.0) 

In [30]:
dummy_input = torch.zeros((1, 4800, 1024))

In [31]:
stb(dummy_input) 

tensor([[[ 0.2312,  0.1291,  0.0728,  ..., -0.0651,  0.0243, -0.0362],
         [ 0.2312,  0.1291,  0.0728,  ..., -0.0651,  0.0243, -0.0362],
         [ 0.2312,  0.1291,  0.0728,  ..., -0.0651,  0.0243, -0.0362],
         ...,
         [ 0.2312,  0.1291,  0.0728,  ..., -0.0651,  0.0243, -0.0362],
         [ 0.2312,  0.1291,  0.0728,  ..., -0.0651,  0.0243, -0.0362],
         [ 0.2312,  0.1291,  0.0728,  ..., -0.0651,  0.0243, -0.0362]]],
       grad_fn=<AddBackward0>)

In [32]:
torch.jit.trace(stb, dummy_input) 

/home/ubuntu/aws_neuron_venv_pytorch/lib/python3.8/site-packages/torch/__init__.py:853: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  assert condition, message
/home/ubuntu/aws_neuron_venv_pytorch/lib/python3.8/site-packages/timm/models/swin_transformer.py:119: TracerWarning: Converting a tensor to a Python integer might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  B = int(windows.shape[0] / (H * W / window_size / window_size))


SwinTransformerBlock(
  original_name=SwinTransformerBlock
  (norm1): LayerNorm(original_name=LayerNorm)
  (attn): WindowAttention(
    original_name=WindowAttention
    (qkv): Linear(original_name=Linear)
    (attn_drop): Dropout(original_name=Dropout)
    (proj): Linear(original_name=Linear)
    (proj_drop): Dropout(original_name=Dropout)
    (softmax): Softmax(original_name=Softmax)
  )
  (drop_path): Identity(original_name=Identity)
  (norm2): LayerNorm(original_name=LayerNorm)
  (mlp): Mlp(
    original_name=Mlp
    (fc1): Linear(original_name=Linear)
    (act): GELU(original_name=GELU)
    (drop1): Dropout(original_name=Dropout)
    (fc2): Linear(original_name=Linear)
    (drop2): Dropout(original_name=Dropout)
  )
)

In [33]:
torch_neuronx.trace(stb, dummy_input)

NeuronModule(original_name=NeuronModule)

In [34]:
swin_transformer = SwinTransformer() 

In [35]:
dummy_input = torch.zeros((1, 3, 224, 224))

In [36]:
swin_transformer(dummy_input) 

tensor([[-8.3674e-01,  8.7812e-01, -2.2818e-01, -1.1928e-01, -1.7823e-02,
          9.7244e-02, -9.4447e-02,  5.2866e-01,  9.6839e-01,  3.7772e-01,
          1.9505e-01,  2.6907e-01, -3.7482e-01,  5.7078e-01,  2.9605e-01,
         -1.0211e-01,  5.4349e-01,  1.1274e+00,  9.0265e-01,  4.0115e-01,
         -7.4968e-02,  1.1743e-01,  6.4910e-01, -2.7082e-01, -3.8752e-01,
          3.7325e-01,  1.1855e+00, -4.5698e-01,  2.4166e-01,  9.5284e-01,
         -7.8355e-02,  2.0212e-01,  4.0976e-01,  2.2871e-01, -3.5197e-01,
         -4.6748e-01, -3.6364e-01,  1.0386e+00,  2.3065e-02,  1.9896e-01,
         -7.2755e-02,  2.9576e-01, -6.8816e-01, -5.9565e-01, -2.6397e-01,
         -4.9731e-01, -1.4363e-01, -4.6121e-01, -4.9600e-01, -6.9411e-01,
         -5.3715e-01, -1.2879e-02, -1.2450e-01,  6.3713e-01,  4.6495e-01,
          2.1782e-01, -4.6260e-01, -1.3102e+00, -1.6793e-01, -1.4850e-01,
         -1.5148e-01,  1.5185e-01, -4.4750e-01, -6.2129e-01,  4.1416e-01,
         -1.1333e-01,  4.5312e-02, -7.

In [37]:
torch.jit.trace(swin_transformer, dummy_input) 

/home/ubuntu/aws_neuron_venv_pytorch/lib/python3.8/site-packages/torch/jit/_trace.py:1001: TracerWarning: Output nr 1. of the traced function does not match the corresponding output of the Python function. Detailed error:
Tensor-likes are not close!

Mismatched elements: 1000 / 1000 (100.0%)
Greatest absolute difference: 1.0012190006673336 at index (0, 409) (up to 1e-05 allowed)
Greatest relative difference: 1479.7131770592641 at index (0, 529) (up to 1e-05 allowed)
  _check_trace(


SwinTransformer(
  original_name=SwinTransformer
  (patch_embed): PatchEmbed(
    original_name=PatchEmbed
    (proj): Conv2d(original_name=Conv2d)
    (norm): LayerNorm(original_name=LayerNorm)
  )
  (pos_drop): Dropout(original_name=Dropout)
  (layers): Sequential(
    original_name=Sequential
    (0): BasicLayer(
      original_name=BasicLayer
      (blocks): ModuleList(
        original_name=ModuleList
        (0): SwinTransformerBlock(
          original_name=SwinTransformerBlock
          (norm1): LayerNorm(original_name=LayerNorm)
          (attn): WindowAttention(
            original_name=WindowAttention
            (qkv): Linear(original_name=Linear)
            (attn_drop): Dropout(original_name=Dropout)
            (proj): Linear(original_name=Linear)
            (proj_drop): Dropout(original_name=Dropout)
            (softmax): Softmax(original_name=Softmax)
          )
          (drop_path): Identity(original_name=Identity)
          (norm2): LayerNorm(original_name=Layer

In [38]:
swin_transformer_neuron = torch_neuronx.trace(swin_transformer, dummy_input) 

In [43]:
# When BasicLayer's depth is 1
from timm.models.swin_transformer import BasicLayer

params = {
    "dim": 16,
    #"out_dim": 32,
    "input_resolution": (80, 240),
    "depth": 1,
    "num_heads": 4,
    #"head_dim": None,
    "window_size": 10,
    "mlp_ratio": 4.0,
    "qkv_bias": True
}
basic_layer = BasicLayer(**params)
dummy_input = torch.zeros((1, params["input_resolution"][0]*params["input_resolution"][1], params["dim"]))
basic_layer_output = basic_layer(dummy_input)
torch.jit.trace(basic_layer, dummy_input)
torch_neuronx.trace(basic_layer, dummy_input)

2023-06-02T12:53:27Z WARNING 42985 [StaticProfiler]: matmul-based transposes inserted by penguin takes up 90.94 percent of all matmul computation


NeuronModule(original_name=NeuronModule)

In [45]:
# when the depth is 2
params["depth"] = 2 # Now testing two sequentially connected BasicLayers
basic_layer = BasicLayer(**params)
dummy_input = torch.zeros((1, params["input_resolution"][0]*params["input_resolution"][1], params["dim"]))
basic_layer_output = basic_layer(dummy_input)
torch.jit.trace(basic_layer, dummy_input)
torch_neuronx.trace(basic_layer, dummy_input)

2023-06-02T12:54:44Z WARNING 43331 [StaticProfiler]: matmul-based transposes inserted by penguin takes up 90.82 percent of all matmul computation


NeuronModule(original_name=NeuronModule)

In [46]:
# the tracing goes right when depth = 4 (or any other number but 2)
params["depth"] = 4 # Now testing two sequentially connected BasicLayers
basic_layer = BasicLayer(**params)
dummy_input = torch.zeros((1, params["input_resolution"][0]*params["input_resolution"][1], params["dim"]))
basic_layer_output = basic_layer(dummy_input)
torch.jit.trace(basic_layer, dummy_input)
torch_neuronx.trace(basic_layer, dummy_input)

2023-06-02T12:55:58Z WARNING 43928 [StaticProfiler]: matmul-based transposes inserted by penguin takes up 87.98 percent of all matmul computation


NeuronModule(original_name=NeuronModule)